In [ ]:
library(BiocManager)
library(BSgenome.Hsapiens.UCSC.hg38)
library(ArchR)
library(ggplot2)
library(TFBSTools)
library(Seurat)
library(ggplot2)
library(dplyr)
library(harmony)
library(BSgenome.Hsapiens.UCSC.hg38)
library(JASPAR2018)
library(edgeR)
data("geneAnnoHg38")
data("genomeAnnoHg38")
geneAnno <- geneAnnoHg38
genomeAnno <- genomeAnnoHg38
addArchRThreads(22)


In [ ]:
pathFragments <- "path to the fragment files "
inputFiles <- list.files(pathFragments, pattern = ".gz$", full.names = TRUE)
names(inputFiles) <- gsub(".fragments.tsv.gz", "", list.files(pathFragments, pattern = ".gz$"))

In [ ]:
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  filterTSS = 6,
  filterFrags = 1500, 
  sampleNames = names(inputFiles),
  geneAnnotation = geneAnno,
  genomeAnnotation = genomeAnno,
  force = TRUE
)

In [ ]:
proj_1<- ArchRProject(
  ArrowFiles = ArrowFiles, 
  geneAnnotation = geneAnno,
  genomeAnnotation = genomeAnno,
  outputDirectory = "path to archr project"
)

proj_1

In [ ]:
proj <- filterDoublets(proj_1)

In [ ]:
proj <- addIterativeLSI(
  ArchRProj = proj, 
  useMatrix = "TileMatrix",iterations = 4,force=TRUE
)


proj <- addUMAP(
  ArchRProj = proj, 
  reducedDims = "IterativeLSI",force=TRUE
)


In [ ]:
proj <- addClusters(input = proj, reducedDims = "IterativeLSI", resolution = 1.5,force=TRUE)

In [ ]:
plotList <- list()
plotList[[1]] <- plotEmbedding(ArchRProj = proj, colorBy = "colData", name = "Sample")
plotList[[2]] <- plotEmbedding(ArchRProj = proj, colorBy = "colData", name = "Clusters", plotParams = list(labelMeans=TRUE))
plotPDF(plotList = plotList, name = "UMAP-Samples-Clusters", width = 6, height = 6, ArchRProj = proj, addDOC = FALSE)
plotList



In [ ]:
req_names <- read.csv('/illumina/scratch/deep_learning/lsundaram/single_cell_datasets/cardiomyogenesis/differentiation_names.csv')
req_names1<-req_names$X
proj<-subsetCells(ArchRProj=proj,cellNames=req_names1)

In [ ]:
coldata<-getCellColData(proj)# Annotation of clusters 
head(coldata)


#
coldata[coldata$Clusters=='C9','Clusters1']='A_iPS'
coldata[coldata$Clusters=='C10','Clusters1']='B_Non Differentiated Cells'
coldata[coldata$Clusters=='C12','Clusters1']='C_Cardiac Mesoderm'
coldata[coldata$Clusters=='C14','Clusters1']='D_Cardiac Progenitor'
coldata[coldata$Clusters=='C13','Clusters1']='E_Non Differentiated Cells'
#
coldata[coldata$Clusters=='C11','Clusters1']='F_Endoderm'
coldata[coldata$Clusters=='C3','Clusters1']='F_Immature Cardiac Fibroblast'
coldata[coldata$Clusters=='C16','Clusters1']='G_Primitive Cardiomyocytes'
coldata[coldata$Clusters=='C15','Clusters1']='H_Mature Cardiomyocytes'
coldata[coldata$Clusters=='C1','Clusters1']='I_Endocardium/Capillaries'
coldata[coldata$Clusters=='C2','Clusters1']='I_Endocardium/Capillaries'
#
coldata[coldata$Clusters=='C6','Clusters1']='J_Epicardium'
coldata[coldata$Clusters=='C4','Clusters1']='J_Epicardium'
coldata[coldata$Clusters=='C5','Clusters1']='L_SMC'
coldata[coldata$Clusters=='C8','Clusters1']='L_SMC'
coldata[coldata$Clusters=='C7','Clusters1']='M_Cardiac Fiborblast'
coldata[(coldata$Sample=='H5_v2') & (coldata$Clusters=='C13') ,'Clusters1']='H_Mature Cardiomyocytes'

proj$Clusters1<-coldata$Clusters1


In [ ]:
#paletteDiscrete
#paletteDiscrete(values = factor, set = "bear", reverse = FALSE)
plotList <- list()
#plotEmbedding(ArchRProj = proj, colorBy = "colData",name = "Sample")
plotList[[1]] <-plotEmbedding(ArchRProj = proj,pal=c("H5_v2"="#E22929","H4"="#FCB31A","EC_v2"="#FF8B74",
                                    "EPC_v2"="#60824F","SMC_v2"="#00CC00","CF_v2"="#00A08A","H3_v2"="#FBDF72",
                                    "H2_v2"="#5BB1CB","H1_v2"="#084081"),colorBy = "colData",name = "Sample")
plotList[[2]] <-plotEmbedding(ArchRProj = proj,pal=c("H5_v2"="#E22929","H4"="#FCB31A","EC_v2"="#FF8B74",
                                    "EPC_v2"="#60824F","SMC_v2"="#00CC00","CF_v2"="#00A08A","H3_v2"="#FBDF72",
                                    "H2_v2"="#5BB1CB","H1_v2"="#084081"),colorBy = "colData",name = "Sample", labelMeans=FALSE)

plotList[[3]] <- plotEmbedding(ArchRProj = proj, pal=c("I_Endocardium/Capillaries"="#E6C2DC","I_Endocardium/Capillaries"="#FF8B74"
                                                       ,"F_Immature Cardiac Fibroblast"="#581845","K_immature Epicardium"="#A2E700",
                                     "L_SMC"="#00CC00","J_Epicardium"="#60824F","M_Cardiac Fiborblast"="#00A08A","L_SMC"="#BED678","A_iPS"="#084081",
                                     "B_Non Differentiated Cells"="#5BB1CB","F_Endoderm"="#B15928",
                                     "C_Cardiac Mesoderm"="#9CDFF0","E_Non Differentiated Cells"="#E1AF00",
                                     "D_Cardiac Progenitor"="#FBDF72","H_Mature Cardiomyocytes"="#E22929","G_Primitive Cardiomyocytes"="#FCB31A"),colorBy = "colData", name = "Clusters1", labelMeans=TRUE)
plotList[[4]] <- plotEmbedding(ArchRProj = proj, pal=c("I_Endocardium/Capillaries"="#E6C2DC","I_Endocardium/Capillaries"="#FF8B74"
                                                       ,"F_Immature Cardiac Fibroblast"="#581845","K_immature Epicardium"="#A2E700",
                                     "L_SMC"="#00CC00","J_Epicardium"="#60824F","M_Cardiac Fiborblast"="#00A08A","L_SMC"="#BED678","A_iPS"="#084081",
                                     "B_Non Differentiated Cells"="#5BB1CB","F_Endoderm"="#B15928",
                                     "C_Cardiac Mesoderm"="#9CDFF0","E_Non Differentiated Cells"="#E1AF00",
                                     "D_Cardiac Progenitor"="#FBDF72","H_Mature Cardiomyocytes"="#E22929","G_Primitive Cardiomyocytes"="#FCB31A"),colorBy = "colData", name = "Clusters1", labelMeans=FALSE)

#plotEmbedding(ArchRProj = proj, pal=ArchRPalettes$bear,colorBy = "colData",name = "Sample")
plotPDF(plotList = plotList, name = "UMAP-Samples-Clusters", width = 6, height = 6, ArchRProj = proj, addDOC = FALSE)
plotList

In [ ]:
#plotting gene scores on umap
proj <- addImputeWeights(ArchRProj = proj)

markerGenes  <- c("POU5F1","NANOG","MESP1","MESP2","ISL1","TBX5","TNNT2","TTN","MYL2","HAND1","HAND2","PECAM1","CD34","CDH5","TCF21",
    "NOTCH1","NOTCH4","DCN","COL1A1","LUM","MYH11","SOX17","FOXA1","TAGLN", "PDGFRB", "MYH6", "MYH7", "MYL7"
  )


#Plot the UMAP Embedding with Marker Genes Overlayed w/ Imputation
plotList <- list()
plotList[[1]] <- plotEmbedding(ArchRProj = proj, colorBy = "GeneScoreMatrix", name = markerGenes, imputeWeights = getImputeWeights(proj))
plotPDF(plotList = plotList, name = "UMAP-Marker-Gene-Scores-w-Imputation", width = 6, height = 6, ArchRProj = proj, addDOC = FALSE)

#Plot the UMAP Embedding with Marker Genes Overlayed w/o Imputation
plotList <- list()
plotList[[1]] <- plotEmbedding(ArchRProj = proj, colorBy = "GeneScoreMatrix", name = markerGenes, imputeWeights = getImputeWeights(proj))
plotPDF(plotList = plotList, name = "UMAP-Marker-Gene-integration-w-Imputation", width = 6, height = 6, ArchRProj = proj, addDOC = FALSE)
plotList

In [ ]:
#getting reproducible peak 
proj <- addGroupCoverages(ArchRProj = proj, groupBy = "Clusters1",force=TRUE)
#Call Reproducible Peaks w/ Macs2 (~5-10 minutes)
proj <- addReproduciblePeakSet(ArchRProj = proj,groupBy = "Clusters1",force=TRUE)
#Add Peak Matrix
proj <- addPeakMatrix(ArchRProj = proj)



In [ ]:
#Identify Marker Peaks
markersPeaks <- markerFeatures(ArchRProj = proj, useMatrix = "PeakMatrix", groupBy = "Clusters1")

#Visualize Markers as a heatmap
heatmapPeaks <- markerHeatmap(
  seMarker = markersPeaks, 
  cutOff = "FDR <= 0.1 & Log2FC >= 1",
  pal = paletteContinuous(set = "blueYellow")
)
plotPDF(heatmapPeaks, name = "Peak-Marker-Heatmap", width = 8, height = 12, ArchRProj = proj, addDOC = FALSE)

In [ ]:
#addMotifAnnotations
proj <- addMotifAnnotations(ArchRProj = proj, motifSet = "cisbp", name = "Motif", force = TRUE)


#Identify Motif Enrichments
enrichMotifs <- peakAnnoEnrichment(
    seMarker = markersPeaks,
    ArchRProj = proj,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )
heatmapEM <- enrichHeatmap(enrichMotifs,pal = paletteContinuous(set = "solarExtra", n = 100))
plotPDF(heatmapEM, name = "Motifs-Enrich-Heatmap", width = 8, height = 10, ArchRProj = proj, addDOC = FALSE)


In [ ]:
#Adding deviations
proj <- addBgdPeaks(proj,force = TRUE)
#Add chromVAR Deviations (~20-25 min if using CisBP Motif Set)
proj <- addDeviationsMatrix(ArchRProj = proj, peakAnnotation = "Motif",threads=15,force = TRUE)

In [ ]:
gene_integration_matrix<-readRDS('load cleaned friedman rna seurat rds')
gene_integration_matrix


In [ ]:
umap_rna<-Embeddings(object = gene_integration_matrix, reduction = "umap")
Idents(gene_integration_matrix) <- "seurat_clusters"


In [ ]:
gene_integration_matrix@meta.data[gene_integration_matrix@meta.data$batch==0,'batch_name']='iPSC'
gene_integration_matrix@meta.data[gene_integration_matrix@meta.data$batch==1,'batch_name']='Cardiac Mesoderm'
gene_integration_matrix@meta.data[gene_integration_matrix@meta.data$batch==2,'batch_name']='Cardiac Progenitors'
gene_integration_matrix@meta.data[gene_integration_matrix@meta.data$batch==3,'batch_name']='Maturing Cardiomyocytes'
gene_integration_matrix@meta.data[gene_integration_matrix@meta.data$batch==4,'batch_name']='Mature Cardiomyocytes'

In [ ]:

proj <- addGeneIntegrationMatrix(
    ArchRProj = proj, 
    useMatrix = "GeneScoreMatrix",
    matrixName = "GeneIntegrationMatrix",
    reducedDims = "IterativeLSI",
    seRNA = gene_integration_matrix,
    addToArrow = TRUE,
    sampleCellsATAC = 70000,
    sampleCellsRNA = 70000,
    groupATAC="Clusters1",
    groupRNA = "seurat_clusters",
    nameCell = "predictedCell_Un",
    nameGroup = "predictedGroup_Un",
    nameScore = "predictedScore_Un",
    force = TRUE
)